## Analyse the results of a Posterior Sampling Routine

###Tests:
* number of observations -> more crudely stepped observation edf-> non-convergence and wider interval ?
* amount of noise -> more diagonal observation edf -> non-convergence and wider interval ?
* proposal std -> higher: worse mixing, shorter burn in, lower: better mixing, longer burn in
* more samples/run: better approximation of the posterior

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import re
from helper.SimulationJob import SimulationJob
from math import sqrt
from helper.InverseProblem import InverseProblem
from scipy.stats import ks_2samp
import seaborn as sns
from os import path
import torch
from tqdm import tqdm

In [ ]:
mpl.style.use("grayscale_adjusted.mplstyle")

Run experiments with inverse problem

In [ ]:
nn_models = np.array(["nn-in-2-out-20-hid-2-n-200-activ-tanh-regul-0.0001-2-soft-False-rng-568-data-e3513ee46f1829cd90d7e07973b5e4f1.json-resol-512-n_tr-64-opt-ADAM-ep-4000-batch-64-tr_time-12.pt"])
synth_datafiles = np.array(["synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy",
"synth-data-lmb-2.0-m-0.5-t_horiz-200-nagents-10000.npy",
"synth-data-lmb-1.0-m-0.1-t_horiz-200-nagents-10000.npy",
"synth-data-lmb-0.5-m-0.5-t_horiz-200-nagents-10000.npy",
"synth-data-lmb-0.1-m-0.1-t_horiz-200-nagents-10000.npy",
"synth-data-lmb-2.0-m-0.0-t_horiz-200-nagents-10000.npy",
"synth-data-lmb-1.0-m-0.5-t_horiz-200-nagents-10000.npy",
"synth-data-lmb-0.5-m-0.1-t_horiz-200-nagents-10000.npy",
"synth-data-lmb-0.5-m-0.0-t_horiz-200-nagents-10000.npy",
"synth-data-lmb-0.1-m-0.5-t_horiz-200-nagents-10000.npy",
"synth-data-lmb-0.1-m-0.0-t_horiz-200-nagents-10000.npy",
"synth-data-lmb-2.0-m-0.1-t_horiz-200-nagents-10000.npy",])

synth_lmbs = [1,2,1,0.5,0.1,2,1,0.5,0.5,0.1,0.1,2]
synth_ms = [0,0.5,0.1,0.5,0.1,0,0.5,0.1,0,0.5,0,0.1]

seeds = [64,81,100,121,144]
n_obs = [25,50,100,200]
noises = [0.01,0.1,1]
proposals = [0.01,0.05,0.1]

for m in range(1): 
    for d in range(1):
        for r in range(1):
            for o in range(len(n_obs)):
                for n in range(len(noises)):
                    for p in range(len(proposals)):
                        ! python experiment-16-inverse-problem-surrogate.py {synth_datafiles[d]} -tl {synth_lmbs[d]} -tm {synth_ms[d]} -o {n_obs[o]} -p {proposals[p]} -b 1000 -s 6000 -w {noises[n]} -r {seeds[r]} -loc models\{nn_models[m]} --show

In [ ]:
#Experiment 0: All possible config for datafile 0 and seed 64
exp0_result_files = np.array(["experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-1.0-n_observations-200-num_rounds-6000-burn_in-1000-proposal--0.1-0.1--initial_sample--1.5--0.2-nn-nn-start-27575102-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-1.0-n_observations-200-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575101-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-1.0-n_observations-200-num_rounds-6000-burn_in-1000-proposal--0.01-0.01--initial_sample--1.5--0.2-nn-nn-start-27575101-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-200-num_rounds-6000-burn_in-1000-proposal--0.1-0.1--initial_sample--1.5--0.2-nn-nn-start-27575101-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-200-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575100-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-200-num_rounds-6000-burn_in-1000-proposal--0.01-0.01--initial_sample--1.5--0.2-nn-nn-start-27575100-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.01-n_observations-200-num_rounds-6000-burn_in-1000-proposal--0.1-0.1--initial_sample--1.5--0.2-nn-nn-start-27575100-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.01-n_observations-200-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575100-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.01-n_observations-200-num_rounds-6000-burn_in-1000-proposal--0.01-0.01--initial_sample--1.5--0.2-nn-nn-start-27575099-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-1.0-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.1-0.1--initial_sample--1.5--0.2-nn-nn-start-27575099-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-1.0-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575099-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-1.0-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.01-0.01--initial_sample--1.5--0.2-nn-nn-start-27575098-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.1-0.1--initial_sample--1.5--0.2-nn-nn-start-27575098-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575098-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.01-0.01--initial_sample--1.5--0.2-nn-nn-start-27575097-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.01-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.1-0.1--initial_sample--1.5--0.2-nn-nn-start-27575097-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.01-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575097-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.01-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.01-0.01--initial_sample--1.5--0.2-nn-nn-start-27575096-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-1.0-n_observations-50-num_rounds-6000-burn_in-1000-proposal--0.1-0.1--initial_sample--1.5--0.2-nn-nn-start-27575096-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-1.0-n_observations-50-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575096-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-1.0-n_observations-50-num_rounds-6000-burn_in-1000-proposal--0.01-0.01--initial_sample--1.5--0.2-nn-nn-start-27575096-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-50-num_rounds-6000-burn_in-1000-proposal--0.1-0.1--initial_sample--1.5--0.2-nn-nn-start-27575095-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-50-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575095-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-50-num_rounds-6000-burn_in-1000-proposal--0.01-0.01--initial_sample--1.5--0.2-nn-nn-start-27575095-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.01-n_observations-50-num_rounds-6000-burn_in-1000-proposal--0.1-0.1--initial_sample--1.5--0.2-nn-nn-start-27575094-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.01-n_observations-50-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575094-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.01-n_observations-50-num_rounds-6000-burn_in-1000-proposal--0.01-0.01--initial_sample--1.5--0.2-nn-nn-start-27575094-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-1.0-n_observations-25-num_rounds-6000-burn_in-1000-proposal--0.1-0.1--initial_sample--1.5--0.2-nn-nn-start-27575094-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-1.0-n_observations-25-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575093-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-1.0-n_observations-25-num_rounds-6000-burn_in-1000-proposal--0.01-0.01--initial_sample--1.5--0.2-nn-nn-start-27575093-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-25-num_rounds-6000-burn_in-1000-proposal--0.1-0.1--initial_sample--1.5--0.2-nn-nn-start-27575093-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-25-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575092-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-25-num_rounds-6000-burn_in-1000-proposal--0.01-0.01--initial_sample--1.5--0.2-nn-nn-start-27575092-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.01-n_observations-25-num_rounds-6000-burn_in-1000-proposal--0.1-0.1--initial_sample--1.5--0.2-nn-nn-start-27575092-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.01-n_observations-25-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575091-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.01-n_observations-25-num_rounds-6000-burn_in-1000-proposal--0.01-0.01--initial_sample--1.5--0.2-nn-nn-start-27575091-seed-64.npz",
])


# Experiment 1 n obs:
for m in range(1): 
    for d in range(len(synth_datafiles)):
        for r in range(len(seeds)):
            for o in [2]:
                for n in [1,]:
                    for p in [1,]:
                        ! python experiment-16-inverse-problem-surrogate.py {synth_datafiles[d]} -tl {synth_lmbs[d]} -tm {synth_ms[d]} -o {n_obs[o]} -p {proposals[p]} -b 1000 -s 6000 -w {noises[n]} -r {seeds[r]} -loc models\{nn_models[m]} --show


In [ ]:
#Experiment 1: 5 reps of one basic config for all datafiles
exp1_result_files = np.array(["experiment-16--synth-data-lmb-0.1-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575137-seed-100.npz",
"experiment-16--synth-data-lmb-2.0-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575141-seed-144.npz",
"experiment-16--synth-data-lmb-2.0-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575140-seed-121.npz",
"experiment-16--synth-data-lmb-2.0-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575140-seed-100.npz",
"experiment-16--synth-data-lmb-2.0-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575140-seed-81.npz",
"experiment-16--synth-data-lmb-2.0-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575139-seed-64.npz",
"experiment-16--synth-data-lmb-0.1-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575139-seed-144.npz",
"experiment-16--synth-data-lmb-0.1-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575138-seed-121.npz",
"experiment-16--synth-data-lmb-0.1-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575137-seed-81.npz",
"experiment-16--synth-data-lmb-0.1-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575136-seed-64.npz",
"experiment-16--synth-data-lmb-0.1-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575136-seed-144.npz",
"experiment-16--synth-data-lmb-0.1-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575136-seed-121.npz",
"experiment-16--synth-data-lmb-0.1-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575135-seed-100.npz",
"experiment-16--synth-data-lmb-0.1-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575135-seed-81.npz",
"experiment-16--synth-data-lmb-0.1-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575135-seed-64.npz",
"experiment-16--synth-data-lmb-0.5-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575134-seed-144.npz",
"experiment-16--synth-data-lmb-0.5-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575134-seed-121.npz",
"experiment-16--synth-data-lmb-0.5-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575134-seed-100.npz",
"experiment-16--synth-data-lmb-0.5-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575133-seed-81.npz",
"experiment-16--synth-data-lmb-0.5-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575133-seed-64.npz",
"experiment-16--synth-data-lmb-0.5-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575133-seed-144.npz",
"experiment-16--synth-data-lmb-0.5-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575132-seed-121.npz",
"experiment-16--synth-data-lmb-0.5-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575132-seed-100.npz",
"experiment-16--synth-data-lmb-0.5-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575132-seed-81.npz",
"experiment-16--synth-data-lmb-0.5-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575131-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575131-seed-144.npz",
"experiment-16--synth-data-lmb-1.0-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575131-seed-121.npz",
"experiment-16--synth-data-lmb-1.0-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575130-seed-100.npz",
"experiment-16--synth-data-lmb-1.0-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575130-seed-81.npz",
"experiment-16--synth-data-lmb-1.0-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575130-seed-64.npz",
"experiment-16--synth-data-lmb-2.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575129-seed-144.npz",
"experiment-16--synth-data-lmb-2.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575129-seed-121.npz",
"experiment-16--synth-data-lmb-2.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575129-seed-100.npz",
"experiment-16--synth-data-lmb-2.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575128-seed-81.npz",
"experiment-16--synth-data-lmb-2.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575128-seed-64.npz",
"experiment-16--synth-data-lmb-0.1-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575128-seed-144.npz",
"experiment-16--synth-data-lmb-0.1-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575128-seed-121.npz",
"experiment-16--synth-data-lmb-0.1-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575127-seed-100.npz",
"experiment-16--synth-data-lmb-0.1-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575127-seed-81.npz",
"experiment-16--synth-data-lmb-0.1-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575126-seed-64.npz",
"experiment-16--synth-data-lmb-0.5-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575126-seed-144.npz",
"experiment-16--synth-data-lmb-0.5-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575125-seed-121.npz",
"experiment-16--synth-data-lmb-0.5-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575125-seed-100.npz",
"experiment-16--synth-data-lmb-0.5-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575124-seed-81.npz",
"experiment-16--synth-data-lmb-0.5-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575124-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575124-seed-144.npz",
"experiment-16--synth-data-lmb-1.0-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575123-seed-121.npz",
"experiment-16--synth-data-lmb-1.0-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575123-seed-100.npz",
"experiment-16--synth-data-lmb-1.0-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575123-seed-81.npz",
"experiment-16--synth-data-lmb-1.0-m-0.1-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575123-seed-64.npz",
"experiment-16--synth-data-lmb-2.0-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575122-seed-144.npz",
"experiment-16--synth-data-lmb-2.0-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575122-seed-121.npz",
"experiment-16--synth-data-lmb-2.0-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575122-seed-100.npz",
"experiment-16--synth-data-lmb-2.0-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575121-seed-81.npz",
"experiment-16--synth-data-lmb-2.0-m-0.5-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575121-seed-64.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575121-seed-144.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575120-seed-121.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575120-seed-100.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575120-seed-81.npz",
"experiment-16--synth-data-lmb-1.0-m-0.0-t_horiz-200-nagents-10000.npy--noise-0.1-n_observations-100-num_rounds-6000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575119-seed-64.npz",
                             ])

In [ ]:
dirname = path.abspath("experiment-data\\")
datafiles = exp1_result_files
files_to_load = range(0,len(datafiles))
# files_to_load = range(0,20)
paths = [path.join(dirname,datafiles[i]) for i in files_to_load]
paths

In [ ]:
print(len(datafiles))

In [ ]:
all_data = [np.load(p) for p in paths]

In [ ]:
in_research = 50
name = datafiles[in_research]
data = all_data[in_research]
name

experiment-16--synth-data-lmb-1-m-0-t_horiz-200-nagents-100000.npy--noise-0.05-n_observations-100-num_rounds-3000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27575052-seed-3324582059.npz
experiment-16--synth-data-lmb-1-m-0-t_horiz-200-nagents-100000.npy--noise-0.05-n_observations-100-num_rounds-3000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27574367-seed-1248087043.npz
experiment-16--synth-data-lmb-1-m-0-t_horiz-200-nagents-100000.npy--noise-0.05-n_observations-100-num_rounds-3000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27574365-seed-230193769.npz
experiment-16--synth-data-lmb-1-m-0-t_horiz-200-nagents-100000.npy--noise-0.05-n_observations-1000-num_rounds-3000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27574364-seed-3357578862.npz


name = "experiment-16--synth-data-lmb-1-m-0-t_horiz-200-nagents-100000.npy--noise-0.05-n_observations-100-num_rounds-3000-burn_in-1000-proposal--0.05-0.05--initial_sample--1.5--0.2-nn-nn-start-27574365-seed-230193769.npz"
datafile = path.join(dirname,name)
data = np.load(datafile)


In [ ]:
x = re.search("(?<=--)(.*?)(?=--)", name)
synth_data_name = x.group()
print(synth_data_name)
#y = re.findall("(?<=-)(.*?)(?=[-/.])",synth_data_name)
#print(y)

In [ ]:
data

In [ ]:
df = pd.DataFrame.from_dict(dict(data),orient='columns')

In [ ]:
df
burnin = 1000
df_postburn = df.loc[burnin:,:]
df_postburn

In [ ]:
plt.figure(1)
plt.subplot(211)
#plt.scatter(df.index,df['lmb'],marker='x')
plt.plot(df.index,df['lmb'])
plt.ylabel("lmb")
plt.subplot(212)
plt.plot(df.index,df['m'])
plt.ylabel("m")
plt.show()


In [ ]:
# from IPython.display import Image
# Image(filename=f"""experiment-data\\inverse-validation-exp5~multiple\\hist-from--{name}-.png""")

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)
lmb_count, lmb_x, _ = axs[0].hist(df_postburn.loc[:,'lmb'],bins=200)
m_count, m_x, _ = axs[1].hist(df_postburn.loc[:,'m'],bins=200)
axs[0].set_ylabel("count")
axs[0].set_xlabel("lambda")
axs[1].set_xlabel("mean opinion")
plt.show()

In [ ]:
m_MAP = float(m_x[np.argmax(m_count)])
lmb_MAP = float(lmb_x[np.argmax(lmb_count)])
print(m_MAP,lmb_MAP)
MAP = dict(m=m_MAP,lmb=lmb_MAP)

In [ ]:
plt.figure()
plt.hist2d(df_postburn.loc[:,'lmb'],df_postburn.loc[:,'m'],density=True)
plt.show()

In [ ]:
sns.pairplot(df_postburn,kind='hist')
plt.show()

In [ ]:
synth_data_path = f"""experiment-data\\{synth_data_name}"""
synth = np.load(synth_data_path)
s_df = pd.DataFrame(synth,columns = ["w"])
s_df

In [ ]:
plt.figure(1)
plt.hist(s_df["w"],bins=200)
plt.show()

In [ ]:
def generate_invariant(lmb,m):
    gamma = 0.01
    theta_std = sqrt(gamma * lmb)
    assert gamma > 0
    experiment_assumptions = dict(
        free_parameters={"lmb", "m"},
        theta_bound=lambda g, w: (1 - g) / (1 + abs(w)),
        gamma=gamma,
        lmb_bound=(1 / (3 * gamma) - 2 / 3 + gamma / 3),
        p=lambda w: 1,
        d=lambda w: (1 - w ** 2),
        t_horiz=200,
        nagents=10000,
    )

    t_horiz = experiment_assumptions["t_horiz"]
    nagents = experiment_assumptions["nagents"]

    # create synthetic data
    job = SimulationJob(
        gamma,
        theta_std,
        experiment_assumptions["theta_bound"],
        experiment_assumptions["p"],
        experiment_assumptions["d"],
        m,
        t_horiz,
        nagents,
        True,
    )

    job.run()
    return job.result

In [ ]:
str(data["nn"])

In [ ]:
model = torch.load(str(data["nn"]))

In [ ]:
for combo in [MAP,]:
    combo["w"] = generate_invariant(combo["lmb"],combo["m"])
    transform_input = lambda x: (x - torch.Tensor([6, 0])) / torch.Tensor([12, 2])
    combo["nn_pred"] = model(transform_input(torch.tensor([combo["lmb"],combo["m"]]))).detach()

In [ ]:
post_samples = torch.tensor(df_postburn.loc[:,["lmb","m"]].sample(1000, axis=0).to_numpy(),dtype=torch.float32)
post_samples
preds = np.transpose(model(transform_input(post_samples)).detach())
mean_pred = torch.mean(preds,axis=0)

In [ ]:
h = np.array(2 /len(MAP["nn_pred"]))
centers = [-1 + h / 2 + i * h for i in range(len(MAP["nn_pred"]))]
preds_norm = preds/h
print(preds.shape)
preds_norm.shape


In [ ]:
plt.figure(4)
plt.hist(s_df["w"],bins=200,histtype = 'step',label="true population",density=True)
plt.hist(MAP["w"],bins=200,histtype='step',label="MAP estimate - simulation",density=True)

plt.plot(centers,MAP["nn_pred"]/h, label="MAP estimate - nn")
assert preds_norm.shape[0] == len(centers)
# plt.boxplot(preds_norm[1:-1,:],positions=centers[1:-1],vert=True)
plt.legend(loc='upper right',bbox_to_anchor =(1.4,1))
plt.show()


In [ ]:
plt.boxplot(preds/h,positions=centers)

### Zoom in 

In [ ]:
start_index = 1600
end_index = 1700

In [ ]:
df_zoom = df.loc[start_index:end_index,:]
plt.figure(2)
plt.subplot(211)
plt.scatter(df_zoom.index,df_zoom.loc[:,'lmb'])
plt.subplot(212)
plt.scatter(df_zoom.index,df_zoom.loc[:,'m'])

### Comparison synthetic population vs synthetic sample

In [ ]:
def generate_synth_sample(synth_population,n_observations,noise_std,seed=None):
    if seed is None:
        seed = random.randint(1, 2**32 - 1)
    else:
        seed = seed

    random.seed(seed)
    np.random.seed(seed)
    observed_data = synth_population[0:n_observations]
    # add noise
    noise_std = noise_std
    noisy_observed_data = InverseProblem.add_noise(observed_data, noise_std)
    return noisy_observed_data

In [ ]:
name

In [ ]:
seed = int(re.search("(?<=seed-)(.*?)(?=[.])",name).group())
noise = float(re.search("(?<=noise-)(.*?)(?=[-])",name).group())
n_observations = int(re.search("(?<=n_observations-)(.*?)(?=[-])",name).group())

In [ ]:
print(seed,noise,n_observations)

In [ ]:
synth_sample = generate_synth_sample(synth,n_observations,noise,seed)
np.mean(synth)

In [ ]:
plt.figure(5)
plt.hist(synth_sample,200)
plt.show()

In [ ]:
def cdf(x,hist):
    hist_arr = np.array(hist)
    cum_hist_arr = hist_arr.cumsum()
    n_bins = len(hist_arr)
    bin_width = 2 / n_bins
    x_bin = np.floor_divide(x + 1, bin_width).astype(int)
    cdf_vals = cum_hist_arr[x_bin]
    return cdf_vals

In [ ]:
def acceptreject(n):
    tries = 0
    s = []
    for i in tqdm(range(n)):
        u = np.random.rand(2)
        tries += 1
        while u[0] > cdf(u[1]*2-1,MAP["nn_pred"]):
            u = np.random.rand(2)
            tries += 1
        s.append(u[1]*2-1)
        i += 1
    print(tries)
    return s
        

In [ ]:
n = acceptreject(100000)
plt.figure()
plt.hist(n,histtype='step',cumulative=True,density=True,label="accept reject",bins=20)
plt.plot(np.linspace(-1,1,300),cdf(np.linspace(-1,1,300),MAP["nn_pred"]),label="nn pred cdf")
plt.legend()

In [ ]:
max(s_df["w"])

In [ ]:
plt.figure(6,figsize=(12,8))
plt.hist(s_df["w"],histtype='step',bins=200,label="true population",cumulative=True,density=True)
plt.hist(synth_sample,histtype = 'step',bins=200,cumulative=True,density=True,label="observed sample")
plt.hist(MAP["w"],bins=200,histtype='step',label="MAP estimate - simulation",cumulative=True,density=True)
plt.plot(np.linspace(-1,1,300),cdf(np.linspace(-1,1,300),MAP["nn_pred"]), label="MAP estimate - nn")
plt.legend()
plt.show()

In [ ]:
ks_dist_MAP_sim = ks_2samp(MAP["w"],synth_sample)
ks_dist_synth = ks_2samp(s_df["w"],synth_sample)
# ks_dist_MAP_nn = ks_2samp(n,synth_sample)
# display(pd.DataFrame(index=["MAP estimate - sim","MAP estimate - nn","true population"],data = [ks_dist_MAP_sim.pvalue,ks_dist_MAP_nn.pvalue,ks_dist_synth.pvalue],columns = ["p-value"]))
display(pd.DataFrame(index=["MAP","synth data"],data = [ks_dist_MAP.pvalue,ks_dist_synth.pvalue],columns = ["p-value"]))